In [43]:
import numpy as np
import pandas as pd

import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt

import tifffile
import av

import tqdm

import gbeflow

In [2]:
fs = ['20180108_htl_glc_sc11_mmzm_rotate_brt',
 '20180108_htl_glc_sc2_mmzm_wp_rotate_brt',
 '20180108_htl_glc_sc9_mmzp_rotate_brt',
 '20180110_htl_glc-CreateImageSubset-01_sc10_wt_rotate_brt',
 '20180110_htl_glc-CreateImageSubset-02_sc11_htl_rotate_brt',
 '20180110_htl_glc_sc14_mmzp_rotate_brt',
 '20180110_htl_glc_sc15_mmzm_rotate_brt',
 '20180110_htl_glc_sc6_mmzm_rotate_brt',
 '20180112_htlglc_tl_sc11_mmzp_rotate_brt',
 '20180112_htlglc_tl_sc4_resille_rotate_brt']

In [3]:
df = pd.read_csv('20181128-tracking.csv')

# Test method on a single sample

In [4]:
f = fs[2]

Open video file

In [5]:
v = av.open(f+'.avi')

Read in each frame in the video

In [6]:
%%time
Larr = []
for packet in v.demux():
    for frame in packet.decode():
        Larr.append(np.asarray(frame.to_image()))

deprecated pixel format used, make sure you did set range correctly


CPU times: user 2.67 s, sys: 545 ms, total: 3.21 s
Wall time: 3.21 s


Save list of arrays with one array per frame in the video

In [7]:
vimg = np.array(Larr)
vimg.shape

(155, 1266, 1266, 3)

Select a subset of the tracks to plot on image

In [8]:
sb = df[df['f']==f]

Settings to optimize output from matplotlib

```python
fig.tight_layout(pad=0)
plt.setp([ax.get_xticklines() + ax.get_yticklines() + ax.get_xgridlines() + ax.get_ygridlines()],antialiased=False)
```

Plot as single timepoint to check that the plot looks good

# Save figure to array/tiff 
https://stackoverflow.com/questions/7821518/matplotlib-save-plot-to-numpy-array
```python
data = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
data = data.reshape(fig.canvas.get_width_height()[::-1] + (3,))
```

Need to call `matplotlib.use('agg')` before calling `import matplotlib.pyplot as plt`

https://stackoverflow.com/questions/35355930/matplotlib-figure-to-image-as-a-numpy-array
```python
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure

fig = Figure()
canvas = FigureCanvas(fig)
ax = fig.gca()

ax.text(0.0,0.0,"Test", fontsize=45)
ax.axis('off')

canvas.draw()       # draw the canvas, cache the renderer

image = np.fromstring(canvas.tostring_rgb(), dtype='uint8')
```

In [15]:
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure

In [26]:
fig,ax = plt.subplots()
ax.imshow(vimg[t])

In [28]:
for t in [10]:
    print(t)

10


In [29]:
for t in [5]:
    fig = Figure()
    canvas = FigureCanvas(fig)
    ax = fig.gca()
    
    ax.imshow(vimg[t])
    ax.scatter(sb[sb.t==t].x,sb[sb.t==t].y,c='r')
    
    canvas.draw()
    
    image = np.frombuffer(canvas.tostring_rgb(),dtype='uint8')

In [19]:
data = image.reshape(fig.canvas.get_width_height()[::-1]+(3,))

In [20]:
data.shape

(480, 640, 3)

In [25]:
tifffile.imsave('testplot.tif',data=data)

# Try on multiple timepoints

In [31]:
vimg.shape

(155, 1266, 1266, 3)

In [45]:
Larr = []
for t in tqdm.tqdm(range(vimg.shape[0])):
    fig = Figure()
    canvas = FigureCanvas(fig)
    ax = fig.gca()
    
    ax.imshow(vimg[t])
    ax.scatter(sb[sb.t==t].x,sb[sb.t==t].y,c='r')
    
    fig.tight_layout(pad=0)
#     fig.axis('off')
    ax.axis('off')
    
    canvas.draw()
    
    image = np.frombuffer(canvas.tostring_rgb(),dtype='uint8')
    data = image.reshape(fig.canvas.get_width_height()[::-1]+(3,))
    
    Larr.append(data)

100%|██████████| 155/155 [00:18<00:00,  7.91it/s]


In [33]:
np.array(Larr).shape

(155, 480, 640, 3)

In [46]:
tifffile.imsave(f+'_tracks.tiff',data=np.array(Larr))

# Test new gbeflow function

In [48]:
from imp import reload

In [50]:
reload(gbeflow)

<module 'gbeflow' from '/Users/morganschwartz/Code/germband-extension/gbeflow/__init__.py'>

In [51]:
gbeflow.make_track_movie(f+'.avi',sb,c='r',name='functiontest')

  1%|          | 1/155 [00:00<00:18,  8.15it/s]

Movie import complete


100%|██████████| 155/155 [00:18<00:00,  6.98it/s]


In [52]:
for f in fs:
    try:
        gbeflow.make_track_movie(f+'.avi',df[df.f==f],c='r',name=f+'_tracks')
    except:
        print('Failed:',f)

  1%|          | 1/155 [00:00<00:18,  8.24it/s]

Movie import complete


100%|██████████| 155/155 [00:16<00:00,  9.22it/s]
deprecated pixel format used, make sure you did set range correctly
 (repeated 464 more times)
Format avi detected only with low score of 1, misdetection possible!
deprecated pixel format used, make sure you did set range correctly


Failed: 20180108_htl_glc_sc2_mmzm_wp_rotate_brt


  1%|          | 1/155 [00:00<00:16,  9.29it/s]

...

Failed: 20180110_htl_glc_sc6_mmzm_rotate_brt


  1%|          | 1/168 [00:00<00:19,  8.42it/s]

Movie import complete


100%|██████████| 168/168 [00:17<00:00,  7.84it/s]
deprecated pixel format used, make sure you did set range correctly
 (repeated 167 more times)
Format avi detected only with low score of 1, misdetection possible!


Failed: 20180112_htlglc_tl_sc4_resille_rotate_brt


# Notes from the track movies generated above

- In some cases the points jump huge distances that are biologically unreasonable and don't seem entirely justified by evaluation of the vector maps by eye
- While waiting for gbe to start, points seem vulerable to being bounced around until they are out of the range of the gbe movement when it actually starts
- Points also can get caught in the edge of the embryo and follow along that border instead of actually following the wave of germband extension
- None of the existing examples appropriately track gbe

# Next steps

- Try introducing stronger smoothing either in the calculation of the vector map or in the interpolation of the map
- Calculate the start of mesoderm invagination as a temporal marker for alignment to ultimately ensure that point interpolation starts at the beginning of gbe